# Лабараторная работа по python № 1

# Автор: Ежов Дмитрий Александрович

Группа: J3111, ИСУ: 471242


In [1]:
# Download dataset: Seoul Bike Sharing Demand
# https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand
%pip install ucimlrepo

# import
import numpy as np
import pandas as pd

from ucimlrepo import fetch_ucirepo

Note: you may need to restart the kernel to use updated packages.


# I. Подготовка данных и базовые операции с NumPy

## 1. Загрузка и подготовка данных:


In [2]:
# Import data, show metadata

# https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand

seoul_bike_sharing_demand = fetch_ucirepo(id=560)
# print(seoul_bike_sharing_demand)

# metadata
print(seoul_bike_sharing_demand.metadata)

# variable information
print(seoul_bike_sharing_demand.variables)


{'uci_id': 560, 'name': 'Seoul Bike Sharing Demand', 'repository_url': 'https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand', 'data_url': 'https://archive.ics.uci.edu/static/public/560/data.csv', 'abstract': 'The dataset contains count of public bicycles rented per hour in the Seoul Bike Sharing System, with corresponding weather data and holiday information', 'area': 'Business', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 8760, 'num_features': 13, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Functioning Day'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Mon Feb 05 2024', 'dataset_doi': '10.24432/C5F62R', 'creators': [], 'intro_paper': None, 'additional_info': {'summary': 'Currently Rental bikes are introduced in many urban cities for the enhancement of mobility comfort. It is important to make the rental bike available and a

In [3]:
# data (as pandas dataframes)
X = seoul_bike_sharing_demand.data.features
y = seoul_bike_sharing_demand.data.targets

print(X)
print(y)


# Nulls. Zero nulls
print(X.isnull().sum())
print(y.isnull().sum())

# Categorical
print(X["Seasons"].astype("category"))
X["Holiday"] = X["Holiday"].map({"No Holiday": 0, "Holiday": 1})
y["Functioning Day"] = y["Functioning Day"].map({"No": 0, "Yes": 1})
X["Seasons"] = X["Seasons"].map({"Winter": 0, "Spring": 1, "Summer": 2, "Autumn": 3})


            Date  Rented Bike Count  Hour  Temperature  Humidity  Wind speed  \
0      1/12/2017                254     0         -5.2        37         2.2   
1      1/12/2017                204     1         -5.5        38         0.8   
2      1/12/2017                173     2         -6.0        39         1.0   
3      1/12/2017                107     3         -6.2        40         0.9   
4      1/12/2017                 78     4         -6.0        36         2.3   
...          ...                ...   ...          ...       ...         ...   
8755  30/11/2018               1003    19          4.2        34         2.6   
8756  30/11/2018                764    20          3.4        37         2.3   
8757  30/11/2018                694    21          2.6        39         0.3   
8758  30/11/2018                712    22          2.1        41         1.0   
8759  30/11/2018                584    23          1.9        43         1.3   

      Visibility  Dew point temperature

/tmp/ipykernel_5592/4189243717.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Holiday"] = X["Holiday"].map({"No Holiday": 0, "Holiday": 1})
/tmp/ipykernel_5592/4189243717.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["Functioning Day"] = y["Functioning Day"].map({"No": 0, "Yes": 1})
/tmp/ipykernel_5592/4189243717.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

## 2. Основные операции с NumPy:


In [4]:
# Create a vector of each feature
feature_columns = {col: np.asarray(X[col]) for col in X.columns}
print(feature_columns["Rented Bike Count"])  # for example
# Create a matrix of features
weather_matrix = X[
    [
        "Temperature",
        "Wind speed",
        "Humidity",
        "Visibility",
        "Dew point temperature",
        "Solar Radiation",
        "Rainfall",
        "Snowfall",
        "Seasons",
    ]
].values
features = np.concatenate((X.values, y.values), axis=1)
print(features)
print(weather_matrix)

[254 204 173 ... 694 712 584]
[['1/12/2017' 254 0 ... 0 0 1]
 ['1/12/2017' 204 1 ... 0 0 1]
 ['1/12/2017' 173 2 ... 0 0 1]
 ...
 ['30/11/2018' 694 21 ... 3 0 1]
 ['30/11/2018' 712 22 ... 3 0 1]
 ['30/11/2018' 584 23 ... 3 0 1]]
[[-5.2  2.2 37.  ...  0.   0.   0. ]
 [-5.5  0.8 38.  ...  0.   0.   0. ]
 [-6.   1.  39.  ...  0.   0.   0. ]
 ...
 [ 2.6  0.3 39.  ...  0.   0.   3. ]
 [ 2.1  1.  41.  ...  0.   0.   3. ]
 [ 1.9  1.3 43.  ...  0.   0.   3. ]]


## 3. Индексация и срезы:


In [5]:
# Get subvector where rented bike count = 0
mask = feature_columns["Rented Bike Count"] == 0
print(feature_columns["Date"][mask][:10])
# Get submatrix with temperature, Wind speed
print(weather_matrix)
print(weather_matrix[:, 0:2])

import re

md = np.vectorize(
    lambda x: bool(re.match(r"\b\d{2}/12/\d{4}\b", x))
)  
maskd = md(feature_columns["Date"])
print(feature_columns["Rented Bike Count"][maskd].mean())
mj = np.vectorize(
    lambda x: bool(re.match(r"\b\d{2}/06/\d{4}\b", x))
)  
maskj = mj(feature_columns["Date"])
print(feature_columns["Rented Bike Count"][maskj].mean())

['11/4/2018' '11/4/2018' '11/4/2018' '11/4/2018' '11/4/2018' '11/4/2018'
 '11/4/2018' '11/4/2018' '11/4/2018' '11/4/2018']
[[-5.2  2.2 37.  ...  0.   0.   0. ]
 [-5.5  0.8 38.  ...  0.   0.   0. ]
 [-6.   1.  39.  ...  0.   0.   0. ]
 ...
 [ 2.6  0.3 39.  ...  0.   0.   3. ]
 [ 2.1  1.  41.  ...  0.   0.   3. ]
 [ 1.9  1.3 43.  ...  0.   0.   3. ]]
[[-5.2  2.2]
 [-5.5  0.8]
 [-6.   1. ]
 ...
 [ 2.6  0.3]
 [ 2.1  1. ]
 [ 1.9  1.3]]
213.1193181818182
1216.648148148148


# I. Операции с векторами и матрицами

## 1. Операции над векторами


In [ ]:
# simple operations
print((feature_columns["Rainfall"] * 10 + feature_columns["Snowfall"]).max())
# broadcasting
print(feature_columns["Temperature"] * feature_columns["Humidity"][:, np.newaxis])

350.0
[-192.4 -209.  -234.  ...  101.4   86.1   81.7]


## 2. Скалярное произведение:


In [7]:
# dot product of vectors
print(feature_columns["Rainfall"].T.dot(feature_columns["Rented Bike Count"]))
print(feature_columns["Temperature"].T.dot(feature_columns["Rented Bike Count"]))
# Temperature has a stronger effect than rain

133299.9
115860790.8


# III. Анализ сходства между векторами

## 1. Корреляция и косинусное сходство:


In [ ]:
# Pearson correlation
def pearson_correlation(x: np.array, y: np.array) -> float:
    if len(x) != len(y):
        raise ValueError("Vectors must be the same length")
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    covariance = np.sum((x - mean_x) * (y - mean_y))
    std_x = np.sqrt(np.sum((x - mean_x) ** 2))
    std_y = np.sqrt(np.sum((y - mean_y) ** 2))

    if std_x == 0 or std_y == 0:
        return 0
    return covariance / (std_x * std_y)


# Verify
from scipy.stats import pearsonr

print(
    f"My pearson correlation between Temperature and Solar Radiation: {pearson_correlation(feature_columns['Temperature'], feature_columns['Solar Radiation'])}\n"
    f"Scipy pearosn correlation: {pearsonr(feature_columns['Temperature'], feature_columns['Solar Radiation'])[0]}",
)

Note: you may need to restart the kernel to use updated packages.
My pearson correlation between Temperature and Solar Radiation: 0.353505470147949
Scipy pearosn correlation: 0.3535054701479491


In [ ]:
# Cosine correlation
def cosine_similarity(x: np.array, y: np.array) -> float:
    if len(x) != len(y):
        raise ValueError("Vectors must be the same length")

    dot_product = np.dot(x, y)

    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)

    if norm_x == 0 or norm_y == 0:
        return 0
    return dot_product / (norm_x * norm_y)


# Verify
from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity

print(
    f"My cosine similary between Temperature and Solar Radiation: {cosine_similarity(feature_columns['Wind speed'], feature_columns['Rainfall'])}\n"
    f"Scipy cosine similary: {sklearn_cosine_similarity([feature_columns['Wind speed']], [feature_columns['Rainfall']])[0][0]}",
)

Note: you may need to restart the kernel to use updated packages.
My cosine similary between Temperature and Solar Radiation: 0.10196663412659339
Scipy cosine similary: 0.10196663412659379


## 2. Нормализация векторов


In [10]:
def normalize(v: np.array) -> np.array:
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm


print(feature_columns["Hour"])
print(normalize(feature_columns["Hour"]))

[ 0  1  2 ... 21 22 23]
[0.         0.000796   0.00159199 ... 0.01671591 0.01751191 0.0183079 ]


## 3. Поиск сходства


In [11]:
def euclidean_distance(vec1: np.array, vec2: np.array) -> float:
    return np.sqrt(np.sum((vec1 - vec2) ** 2))


def find_most_similar_days(
    query_vector: np.array, data_vectors: np.array, k=3
) -> list[np.array]:
    distances = []

    for i, data_vector in enumerate(data_vectors):
        distance = euclidean_distance(query_vector, data_vector)
        distances.append((distance, i))  

    sorted_distances = sorted(distances, key=lambda x: x[0])

    return [index for _, index in sorted_distances[:k]]


most_sim_days = find_most_similar_days(features[0][1:], features[:, 1:])

for i in most_sim_days:
    print(f"Day index: {i}")
    print(features[i])

Day index: 0
['1/12/2017' 254 0 -5.2 37 2.2 2000 -17.6 0.0 0.0 0.0 0 0 1]
Day index: 655
['28/12/2017' 257 7 -5.0 31 0.5 2000 -19.5 0.0 0.0 0.0 0 0 1]
Day index: 631
['27/12/2017' 262 7 -10.1 38 2.3 2000 -21.7 0.0 0.0 0.0 0 0 1]


# IV. Прогнозирование и кластеризация

## 1. Фильтрация временных рядов:
